In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install accelerate>=0.12.0
!pip install datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 req

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/rebuttal

Mounted at /content/drive
/content/drive/MyDrive/rebuttal


In [ ]:
import json

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from datasets import load_dataset
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_entities(query):
    query = clean(query)
    entities = []
    relations = []
    all_good = []
    words = query.split(" ")
    for word in words:
        if word.startswith("orkg"):
            all_good.append(word)

    for word in all_good:
        if word.startswith("orkgp:"):
            relations.append(word)
        else:
            entities.append(word)

    return {"entities": entities, "relations": relations}

prefix = "translate English to Sparql: "
tokenizer = AutoTokenizer.from_pretrained("en2sparql_T5_model")
model = AutoModelForSeq2SeqLM.from_pretrained("en2sparql_T5_model").to(device)

# books = load_dataset("json", data_files={'test':'test.json'})
books = load_dataset("orkg/SciQA")
# books = load_dataset("awalesushil/DBLP-QuAD")
print(books["test"])

queries = []
sparql = []

for feature in books["test"]:
    ents = get_entities(feature["query"]["sparql"])
    query = prefix + feature.get("question").get("string") + "\nentities: " + str(ent.get("entities")) + "\nrelations: " + str(ents.get("relations"))
    queries.append(query)
    gold_sparql = feature.get("query").get("sparql")
    sparql.append(gold_sparql)

print(len(queries))

def divide_chunks(l_, n_):
    # looping till length l
    for i_ in range(0, len(l_), n_):
        yield l_[i_:i_ + n_]

n = 10

q = list(divide_chunks(queries, n))

gs = []
gst = []
i = 0

for group in q:
    print(str(i)+"%", end="  ")
    i += 0.5
    inputs = tokenizer(group, max_length=512, truncation=True, return_tensors='pt', padding=True).to(device)
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_new_tokens=512, do_sample=True, top_k=30, top_p=0.95)

    generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    generated_texts2 = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)

    generated_texts2 = [x.replace("<pad>", "").replace("</s>", "").strip() for x in generated_texts2]

    gs += generated_texts
    gst += generated_texts2

result = {"questions": queries, "sparql": sparql, "generated_sparql": gs, "generated_with_special_tokens": gst}

with open("ft_T5_results.json", "w", encoding="utf-8") as text_file:
    print(json.dumps(result), file=text_file)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'entities', 'relations', 'temporal', 'held_out'],
    num_rows: 2000
})
2000
0%  2%  4%  6%  8%  10%  12%  14%  16%  18%  20%  22%  24%  26%  28%  30%  32%  34%  36%  38%  40%  42%  44%  46%  48%  50%  52%  54%  56%  58%  60%  62%  64%  66%  68%  70%  72%  74%  76%  78%  80%  82%  84%  86%  88%  90%  92%  94%  96%  98%  100%  102%  104%  106%  108%  110%  112%  114%  116%  118%  120%  122%  124%  126%  128%  130%  132%  134%  136%  138%  140%  142%  144%  146%  148%  150%  152%  154%  156%  158%  160%  162%  164%  166%  168%  170%  172%  174%  176%  178%  180%  182%  184%  186%  188%  190%  192%  194%  196%  198%  200%  202%  204%  206%  208%  210%  212%  214%  216%  218%  220%  222%  224%  226%  228%  230%  232%  234%  236%  238%  240%  242%  244%  246%  248%  250%  252%  254%  256%  258%  260%  262%  264%  266%  268%  270%  272%  274%  276%  278%  280%  282%  284%  286%  288%  290